In [ ]:
import requests
import json
import os
import time
import sys # त्रुटि होने पर एक्ज़िट करने के लिए sys मॉड्यूल इम्पोर्ट किया गया

# ==================== CONFIGURATION ====================

# ⚠️ Unity Catalog Model Configuration
UC_CATALOG_NAME = "workspace" 
UC_SCHEMA_NAME = "ml"
MODEL_NAME_UC = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.house_price_model_uc" 

# 🎯 Serving Endpoint Configuration
ENDPOINT_NAME = "house-price-rf-endpoint"
# Endpoint के लिए वर्कलोड साइज़ (Standard, Small, Medium, Large)
WORKLOAD_SIZE = "Small"
# थ्रूपुट के लिए स्केलिंग (Auto, Fixed)
SCALE_TO_ZERO = True # Idle होने पर endpoint को बंद कर दें (लागत बचाने के लिए)

# 🔑 Authentication and Databricks Environment Details
# यह टोकन और URL Databricks द्वारा स्वचालित रूप से उपलब्ध कराए जाते हैं
DATABRICKS_HOST = os.environ.get("DATABRICKS_HOST")
DATABRICKS_TOKEN = os.environ.get("DATABRICKS_TOKEN")

# 💡 यदि आप इसे Databricks वातावरण के बाहर चला रहे हैं,
# 💡 तो नीचे दी गई पंक्तियों से # हटाएँ और अपने मान भरें।
# DATABRICKS_HOST = "adb-<आपका-instance>.azuredatabricks.net" 
# DATABRICKS_TOKEN = "dapi..." 

if not DATABRICKS_HOST or not DATABRICKS_TOKEN:
    print("❌ त्रुटि: DATABRICKS_HOST या DATABRICKS_TOKEN एनवायरनमेंट वैरिएबल उपलब्ध नहीं हैं।")
    print("कृपया सुनिश्चित करें कि आप इस स्क्रिप्ट को Databricks वातावरण के भीतर चला रहे हैं,")
    print("या मैन्युअल कॉन्फ़िगरेशन के लिए DATABRICKS_HOST और DATABRICKS_TOKEN सेट करें।")
    sys.exit(1)

# Databricks Serving API URL
SERVING_API_URL = f"https://{DATABRICKS_HOST}/api/2.0/serving-endpoints"

# ==================== UTILITY FUNCTIONS ====================

def get_latest_model_version(model_name: str):
    """
    MLflow API का उपयोग करके Unity Catalog में मॉडल का नवीनतम संस्करण प्राप्त करता है।
    """
    try:
        import mlflow
        from mlflow.tracking import MlflowClient
        
        # सुनिश्चित करें कि क्लाइंट UC को इंगित कर रहा है
        client = MlflowClient()
        
        # Unity Catalog में get_latest_versions सीधे काम नहीं करता है, इसलिए search_model_versions का उपयोग करें
        filter_string = f"name = '{model_name}'"
        versions = client.search_model_versions(filter_string=filter_string, order_by=["version DESC"])
        
        if versions:
            latest_version = versions[0].version
            print(f"✅ UC मॉडल '{model_name}' का नवीनतम संस्करण पाया गया: v{latest_version}")
            return latest_version
        else:
            print(f"❌ मॉडल '{model_name}' के लिए कोई संस्करण नहीं मिला।")
            return None
    except Exception as e:
        print(f"❌ मॉडल संस्करण प्राप्त करने में त्रुटि: {e}")
        return None

def create_or_update_endpoint(model_name: str, endpoint_name: str, latest_version: str):
    """
    Serving Endpoint बनाता है या नवीनतम मॉडल संस्करण के साथ उसे अपडेट करता है।
    """
    headers = {
        "Authorization": f"Bearer {DATABRICKS_TOKEN}",
        "Content-Type": "application/json"
    }
    
    # मॉडल डिप्लॉयमेंट के लिए कॉन्फ़िगरेशन
    config_payload = {
        "served_models": [
            {
                "name": f"{endpoint_name}-prod", # Served model का नाम
                "model_name": model_name,
                "model_version": latest_version,
                "workload_size": WORKLOAD_SIZE,
                "scale_to_zero_enabled": SCALE_TO_ZERO
            }
        ]
    }
    
    # मुख्य एंडपॉइंट कॉन्फ़िगरेशन
    endpoint_payload = {
        "name": endpoint_name,
        "config": config_payload
    }

    print(f"\n⏳ Serving Endpoint '{endpoint_name}' की स्थिति की जाँच कर रहे हैं...")

    # 1. एंडपॉइंट मौजूद है या नहीं, यह जांचें (GET)
    response = requests.get(f"{SERVING_API_URL}/{endpoint_name}", headers=headers)

    if response.status_code == 200:
        # एंडपॉइंट मौजूद है -> UPDATE करें
        current_config = response.json().get("config", {})
        current_served_models = current_config.get("served_models", [])
        
        # जाँच करें कि क्या नवीनतम संस्करण पहले से ही डिप्लॉय किया गया है
        is_up_to_date = False
        if current_served_models and current_served_models[0].get("model_version") == latest_version:
            is_up_to_date = True

        if is_up_to_date:
            print(f"⏭️ Endpiont पहले से ही मॉडल संस्करण v{latest_version} के साथ अप-टू-डेट है। कोई कार्रवाई नहीं की गई।")
            return True

        print(f"🔄 एंडपॉइंट मौजूद है। मॉडल को संस्करण v{latest_version} के साथ अपडेट कर रहे हैं...")
        
        update_response = requests.put(
            f"{SERVING_API_URL}/{endpoint_name}/config", 
            headers=headers, 
            data=json.dumps(config_payload)
        )
        
        if update_response.status_code in [200, 202]:
            print(f"✅ अपडेट अनुरोध सफलतापूर्वक भेजा गया। एंडपॉइंट अब नए मॉडल को डिप्लॉय कर रहा है।")
            return poll_deployment_status(endpoint_name, headers)
        else:
            print(f"❌ एंडपॉइंट अपडेट करने में त्रुटि ({update_response.status_code}): {update_response.text}")
            return False

    elif response.status_code == 404:
        # एंडपॉइंट मौजूद नहीं है -> CREATE करें
        print(f"➕ एंडपॉइंट मौजूद नहीं है। नया एंडपॉइंट '{endpoint_name}' बना रहे हैं...")
        
        create_response = requests.post(
            SERVING_API_URL, 
            headers=headers, 
            data=json.dumps(endpoint_payload)
        )
        
        if create_response.status_code == 200:
            print(f"✅ एंडपॉइंट निर्माण अनुरोध सफलतापूर्वक भेजा गया। डिप्लॉयमेंट की प्रतीक्षा कर रहे हैं।")
            return poll_deployment_status(endpoint_name, headers)
        else:
            print(f"❌ एंडपॉइंट बनाने में त्रुटि ({create_response.status_code}): {create_response.text}")
            return False
    else:
        print(f"❌ एंडपॉइंट स्थिति की जाँच करने में अप्रत्याशित त्रुटि ({response.status_code}): {response.text}")
        return False

def poll_deployment_status(endpoint_name: str, headers: dict):
    """
    डिप्लॉयमेंट पूरा होने तक एंडपॉइंट की स्थिति की निगरानी करता है।
    """
    max_retries = 30 # लगभग 5 मिनट
    retry_delay = 10 # 10 सेकंड
    
    print("\n⏳ डिप्लॉयमेंट स्थिति की निगरानी शुरू कर रहे हैं (अधिकतम 5 मिनट)...")

    for i in range(max_retries):
        status_response = requests.get(f"{SERVING_API_URL}/{endpoint_name}", headers=headers)
        
        if status_response.status_code != 200:
            print(f"❌ स्थिति जाँचने में त्रुटि ({status_response.status_code}): {status_response.text}")
            return False

        endpoint_status = status_response.json().get("state", {}).get("config_update_status")
        
        if endpoint_status == "UPDATING":
            print(f"   [{i*retry_delay}s] डिप्लॉयमेंट प्रगति पर है...")
        elif endpoint_status == "UPDATE_SUCCESS":
            print("\n🎉 डिप्लॉयमेंट सफल रहा!")
            print(f"   सर्विंग एंडपॉइंट URL: {SERVING_API_URL.replace('/api/2.0/serving-endpoints', '')}/serving-endpoints/{endpoint_name}/invocations")
            return True
        elif endpoint_status == "UPDATE_FAILED":
            print("\n❌ डिप्लॉयमेंट विफल रहा। विवरण के लिए Databricks UI की जाँच करें।")
            return False
        
        # यदि एंडपॉइंट नया बना है और अभी भी 'NOT_UPDATING' स्थिति में है, तो प्रतीक्षा करें
        elif endpoint_status == "NOT_UPDATING" and i > 0:
            print("\n🎉 एंडपॉइंट सफलतापूर्वक तैयार है।")
            return True
        
        time.sleep(retry_delay)

    print("\n⚠️ डिप्लॉयमेंट टाइमआउट हो गया। कृपया Databricks UI में मैन्युअल रूप से जाँच करें।")
    return False

# ==================== MAIN EXECUTION ====================

if __name__ == "__main__":
    
    print("=" * 70)
    print("🚀 UC मॉडल को DATABRICKS SERVING ENDPOINT पर डिप्लॉय करना")
    print("=" * 70)
    print(f"UC मॉडल: {MODEL_NAME_UC}")
    print(f"एंडपॉइंट: {ENDPOINT_NAME}")
    
    # 1. मॉडल का नवीनतम संस्करण प्राप्त करें
    latest_version = get_latest_model_version(MODEL_NAME_UC)
    
    if latest_version:
        # 2. एंडपॉइंट बनाएं या अपडेट करें
        success = create_or_update_endpoint(MODEL_NAME_UC, ENDPOINT_NAME, latest_version)
        
        if success:
            print("\n✅ डिप्लॉयमेंट प्रक्रिया पूरी हुई।")
        else:
            print("\n❌ डिप्लॉयमेंट प्रक्रिया विफल रही। कृपया लॉग और अनुमतियाँ जाँचें।")
    else:
        print("\n❌ डिप्लॉयमेंट रद्द किया गया क्योंकि कोई वैध मॉडल संस्करण नहीं मिला।")
